In [91]:
import requests
import json
import time
import pickle
import folium
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon
# import censusdata
from sklearn.metrics.pairwise import haversine_distances
from sklearn.cluster import AgglomerativeClustering
import pandas as pd
import math
from bs4 import BeautifulSoup as BS
import ast
from itertools import combinations
from collections import Counter
#from scipy.cluster.hierarchy 
from pyproj import Geod
from shapely.geometry import Point, LineString

%matplotlib inline

In [92]:
hidden = pd.read_pickle('../data/API_key.pkl')
api_key = hidden.loc[0][1]

In [93]:
davidson_pred_black = gpd.read_file('../data/davidson_race_by_tract/davidson_tracts_pred_black.geojson')
davidson_pred_black.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 33 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   geoid       25 non-null     object  
 1   name        25 non-null     object  
 2   total_pop   25 non-null     float64 
 3   total_pop_  25 non-null     float64 
 4   white_alon  25 non-null     float64 
 5   white_al_1  25 non-null     float64 
 6   black_afri  25 non-null     float64 
 7   black_af_1  25 non-null     float64 
 8   nativeamer  25 non-null     float64 
 9   nativeam_1  25 non-null     float64 
 10  asian_alon  25 non-null     float64 
 11  asian_al_1  25 non-null     float64 
 12  hawaiian_p  25 non-null     float64 
 13  hawaiian_1  25 non-null     float64 
 14  other_alon  25 non-null     float64 
 15  other_al_1  25 non-null     float64 
 16  two_or_mor  25 non-null     float64 
 17  two_or_m_1  25 non-null     float64 
 18  two_includ  25 non-null     float64 
 19  tw

In [305]:
all_types = ['gas_station',
             'supermarket',
             'hospital',
             'clothing_store',
             'restaurant',
             'bar',
            'store'] 

In [216]:
# Modified from https://python.gotrained.com/google-places-api-extracting-location-data-reviews/

def search_places_by_coordinate(location, radius, types, api_key, sleep_sec = 2):
    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    places = []
    params = {
        'location': location,
        'radius': radius,
        'types': types,
        'key': api_key
    }
    res = requests.get(endpoint_url, params = params)
    results =  res.json()
    places.extend(results['results'])
    time.sleep(sleep_sec)
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.get(endpoint_url, params = params)
        results = res.json()
        places.extend(results['results'])
        time.sleep(sleep_sec)
    return places

## Because the Google API only lets you search for 1 type at a time, I've used a for loop to iterate through each type I need and create a list of those lists (nested lists, yay!)

In [328]:
ct10105 = []
ct10105_list = []
latitude10105 = []
longitude10105 = []
name10105 = []
type10105 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct10105_res = search_places_by_coordinate(location_10105, radius_10105, i, api_key) # running the API function through each type
    ct10105.append(ct10105_res) # creating a lists of the lists of each type of business
    for list in ct10105:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude10105.append(element['geometry']['location']['lat'])
            longitude10105.append(element['geometry']['location']['lng'])
            name10105.append(element['name'])
            type10105.append(element['types'][0])

In [311]:
len(ct10105) # this should be 7 because we have 7 business types

7

In [313]:
len(ct10105[4]) # This would be the number of gas stations since that is the first type we iterated through

11

In [284]:
print(ct10105) #just testing to see what it looks like

[[{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 36.2520693, 'lng': -86.8297597}, 'viewport': {'northeast': {'lat': 36.2534926302915, 'lng': -86.8284251697085}, 'southwest': {'lat': 36.2507946697085, 'lng': -86.8311231302915}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/gas_station-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/gas_pinlet', 'name': 'Shell', 'photos': [{'height': 4128, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/117265797567492229445">Rooster Davis</a>'], 'photo_reference': 'Aap_uEBidbPXVEOm2F5xL8yHFOhZwUJ2dD35n-0hPDcnleL51qaieJ28XWQV6mbe5d1xqZvG6C3FtRZNVb-UFcY6E4dMS2S9es8E4Q2uZgP0p9HeKW990DWhSF_78X4rHXjbvKjwLeWNt6dyhHqhF9ljqbDuxQPfmVahVAvHgSiIc_ECG2wS', 'width': 3096}], 'place_id': 'ChIJBQHTXnBeZIgR4BxNPRfisdI', 'plus_code': {'compound_code': '752C+R3 Nashville, TN, USA', 'global_code': '868M752C+R3'}, 'price_level': 2, 'rat

In [329]:
type10105

['gas_station',
 'gas_station',
 'gas_station',
 'convenience_store',
 'gas_station',
 'gas_station',
 'convenience_store',
 'gas_station',
 'gas_station',
 'gas_station',
 'convenience_store',
 'gas_station',
 'gas_station',
 'convenience_store',
 'supermarket',
 'grocery_or_supermarket',
 'gas_station',
 'gas_station',
 'gas_station',
 'convenience_store',
 'gas_station',
 'gas_station',
 'convenience_store',
 'supermarket',
 'grocery_or_supermarket',
 'gas_station',
 'gas_station',
 'gas_station',
 'convenience_store',
 'gas_station',
 'gas_station',
 'convenience_store',
 'supermarket',
 'grocery_or_supermarket',
 'lodging',
 'supermarket',
 'point_of_interest',
 'clothing_store',
 'point_of_interest',
 'gas_station',
 'gas_station',
 'gas_station',
 'convenience_store',
 'gas_station',
 'gas_station',
 'convenience_store',
 'supermarket',
 'grocery_or_supermarket',
 'lodging',
 'supermarket',
 'point_of_interest',
 'clothing_store',
 'point_of_interest',
 'meal_takeaway',
 'gas_st

In [330]:
df10105 = pd.DataFrame(zip(name10105, type10105, latitude10105, longitude10105),
               columns =['name', 'type', 'latitude', 'longitude'])
df10105

,name,type,latitude,longitude
0,Shell,gas_station,36.252069,-86.829760
1,Cenex KCK's,gas_station,36.279847,-86.873683
2,Kroger Fuel Center,gas_station,36.210674,-86.838564
3,Brenda's Corner Market,convenience_store,36.280015,-86.873743
4,Kwik Sak,gas_station,36.218140,-86.836695
...,...,...,...,...
135,Smily,convenience_store,36.281132,-86.875341
136,Dottie Wotties Botique,point_of_interest,36.214713,-86.837126
137,"Joelton Hardware, Feed and Farmacy",hardware_store,36.281279,-86.875705
138,Smiley's Liquors,liquor_store,36.210481,-86.837606


In [331]:
df10105.drop_duplicates(inplace=True, ignore_index=True)
df10105

,name,type,latitude,longitude
0,Shell,gas_station,36.252069,-86.829760
1,Cenex KCK's,gas_station,36.279847,-86.873683
2,Kroger Fuel Center,gas_station,36.210674,-86.838564
3,Brenda's Corner Market,convenience_store,36.280015,-86.873743
4,Kwik Sak,gas_station,36.218140,-86.836695
5,Citgo,gas_station,36.279806,-86.873688
6,Friendly Corner Market,convenience_store,36.279847,-86.873683
7,Family Dollar,supermarket,36.211460,-86.839407
8,Tony's Foodland,grocery_or_supermarket,36.280344,-86.876424
9,Fontanel,lodging,36.256853,-86.829741


In [332]:
df10105['tract'] = 'census_tract_10105'
df10105

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.252069,-86.829760,census_tract_10105
1,Cenex KCK's,gas_station,36.279847,-86.873683,census_tract_10105
2,Kroger Fuel Center,gas_station,36.210674,-86.838564,census_tract_10105
3,Brenda's Corner Market,convenience_store,36.280015,-86.873743,census_tract_10105
4,Kwik Sak,gas_station,36.218140,-86.836695,census_tract_10105
5,Citgo,gas_station,36.279806,-86.873688,census_tract_10105
6,Friendly Corner Market,convenience_store,36.279847,-86.873683,census_tract_10105
7,Family Dollar,supermarket,36.211460,-86.839407,census_tract_10105
8,Tony's Foodland,grocery_or_supermarket,36.280344,-86.876424,census_tract_10105
9,Fontanel,lodging,36.256853,-86.829741,census_tract_10105


In [333]:
test_map = folium.Map([36.24885, -86.85388])
for row_index, row_values in df10105.iterrows():
    loc = [row_values['latitude'], row_values['longitude']]
    tool = [str(row_values['name']), str(row_values['type'])]
    
    marker = folium.Marker(
        location = loc, 
        tooltip = tool) 

    marker.add_to(test_map)
test_map

# Now that I've created the DataFrame for the first census tract, I will do the same for the others

In [291]:
location_10106 = '36.25371, -86.80361'
radius_10106 = '4000'

In [292]:
ct10106 = []
ct10106_list = []
latitude10106 = []
longitude10106 = []
name10106 = []
type10106 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct10106_res = search_places_by_coordinate(location_10106, radius_10106, i, api_key) # running the API function through each type
    ct10106.append(ct10106_res) # creating a lists of the lists of each type of business
    for list in ct10106:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude10106.append(element['geometry']['location']['lat'])
            longitude10106.append(element['geometry']['location']['lng'])
            name10106.append(element['name'])
            type10106.append(element['types'][0])

In [293]:
name10106

['Shell',
 'Tri-Star Energy',
 'Shell',
 'Citgo',
 "TERRY'S",
 'Marathon',
 'Little Barn Market',
 'A a Mart',
 'Shell',
 'Tri-Star Energy',
 'Shell',
 'Citgo',
 "TERRY'S",
 'Marathon',
 'Little Barn Market',
 'A a Mart',
 'Family Dollar',
 'Shell',
 'Tri-Star Energy',
 'Shell',
 'Citgo',
 "TERRY'S",
 'Marathon',
 'Little Barn Market',
 'A a Mart',
 'Family Dollar',
 'Nashville',
 'Fontanel',
 'Walmart Supercenter',
 'Whites Creek High School',
 'Carl Black Chevy Woods Amphitheater',
 'Super 8 by Wyndham Whites Creek/ Nashville NW Area',
 'Coinstar',
 'Cedarwood Weddings',
 'Bates Nursery and Garden Center',
 'Parkwood Villa Apartments',
 'Davidson Academy',
 'Dollar General',
 'Workwear Outfitters',
 'Cintas Facility Services',
 'Engle Erin T OD',
 'Dr Jay Kavanagh & Associates',
 "Lowe's Home Improvement",
 'UPS Customer Center',
 'Whites Creek Asphalt',
 "Talbot's Corner",
 'Nursing Home',
 'Ewing Park Junior High School',
 'Skyline Village Apartments',
 'Fleetco',
 'AllParts Medica

In [334]:
df10106 = pd.DataFrame((zip(name10106, type10106, latitude10106, longitude10106)),
               columns =['name', 'type', 'latitude', 'longitude'])

In [335]:
df10106.drop_duplicates(inplace=True, ignore_index=True)
df10106

,name,type,latitude,longitude
0,Shell,gas_station,36.252069,-86.829760
1,Tri-Star Energy,convenience_store,36.276629,-86.795924
2,Shell,gas_station,36.277810,-86.796850
3,Citgo,gas_station,36.232554,-86.781664
4,TERRY'S,convenience_store,36.232562,-86.781676
...,...,...,...,...
111,Misfits of Vintage,book_store,36.278200,-86.798946
112,MTA Distributors,home_goods_store,36.280674,-86.796499
113,Real Wood Store llc,furniture_store,36.279266,-86.798887
114,LibertyX Bitcoin Cashier,atm,36.232999,-86.781412


In [336]:
df10106['tract'] = 'census_tract_10106'
df10106

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.252069,-86.829760,census_tract_10106
1,Tri-Star Energy,convenience_store,36.276629,-86.795924,census_tract_10106
2,Shell,gas_station,36.277810,-86.796850,census_tract_10106
3,Citgo,gas_station,36.232554,-86.781664,census_tract_10106
4,TERRY'S,convenience_store,36.232562,-86.781676,census_tract_10106
...,...,...,...,...,...
111,Misfits of Vintage,book_store,36.278200,-86.798946,census_tract_10106
112,MTA Distributors,home_goods_store,36.280674,-86.796499,census_tract_10106
113,Real Wood Store llc,furniture_store,36.279266,-86.798887,census_tract_10106
114,LibertyX Bitcoin Cashier,atm,36.232999,-86.781412,census_tract_10106


In [337]:
location_10903 = '36.24932, -86.76382'
radius_10903 = '2716'

In [338]:
ct10903 = []
ct10903_list = []
latitude10903 = []
longitude10903 = []
name10903 = []
type10903 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct10903_res = search_places_by_coordinate(location_10903, radius_10903, i, api_key) # running the API function through each type
    ct10903.append(ct10903_res) # creating a lists of the lists of each type of business
    for list in ct10903:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude10903.append(element['geometry']['location']['lat'])
            longitude10903.append(element['geometry']['location']['lng'])
            name10903.append(element['name'])
            type10903.append(element['types'][0])

In [339]:
name10903

['Exxon',
 'MAPCO',
 'Shell',
 'Citgo',
 'Shell',
 'Murphy Express',
 'Exxon',
 'Saint George Market',
 'Dickerson Road Citgo',
 "TERRY'S",
 'Little Barn Market',
 'Exxon',
 'MAPCO',
 'Shell',
 'Citgo',
 'Shell',
 'Murphy Express',
 'Exxon',
 'Saint George Market',
 'Dickerson Road Citgo',
 "TERRY'S",
 'Little Barn Market',
 'H G Hill Food Stores',
 'Family Dollar',
 'Exxon',
 'MAPCO',
 'Shell',
 'Citgo',
 'Shell',
 'Murphy Express',
 'Exxon',
 'Saint George Market',
 'Dickerson Road Citgo',
 "TERRY'S",
 'Little Barn Market',
 'H G Hill Food Stores',
 'Family Dollar',
 'TriStar Skyline Medical Center',
 'SunCrest Home Health',
 'DaVita Briley Parkway Dialysis',
 'Capstone Pediatrics Skyline',
 'Skyline medical centrr',
 'Skyline Medical Center',
 'Skyline medical Plaza',
 'Nashville Neurosurgery Associates',
 'TRISTACA HYDE SLP',
 'Hospital tgt',
 'Exxon',
 'MAPCO',
 'Shell',
 'Citgo',
 'Shell',
 'Murphy Express',
 'Exxon',
 'Saint George Market',
 'Dickerson Road Citgo',
 "TERRY'S",
 

In [341]:
df10903 = pd.DataFrame(zip(name10903, type10903, latitude10903, longitude10903),
               columns =['name', 'type', 'latitude', 'longitude'])

In [342]:
df10903.drop_duplicates(inplace=True, ignore_index=True)
df10903

,name,type,latitude,longitude
0,Exxon,gas_station,36.262900,-86.757836
1,MAPCO,gas_station,36.229043,-86.758997
2,Shell,gas_station,36.268857,-86.755242
3,Citgo,gas_station,36.232554,-86.781664
4,Shell,gas_station,36.242914,-86.736170
...,...,...,...,...
113,Dollar Tree,grocery_or_supermarket,36.245239,-86.762039
114,Merchants Metals,point_of_interest,36.254805,-86.758451
115,Staley Bus Sales / Staley Coach,car_dealer,36.268733,-86.754563
116,Discount Mufflers & Custom Exhaust,car_repair,36.254335,-86.758066


In [343]:
df10903['tract'] = 'census_tract_10903'
df10903

,name,type,latitude,longitude,tract
0,Exxon,gas_station,36.262900,-86.757836,census_tract_10903
1,MAPCO,gas_station,36.229043,-86.758997,census_tract_10903
2,Shell,gas_station,36.268857,-86.755242,census_tract_10903
3,Citgo,gas_station,36.232554,-86.781664,census_tract_10903
4,Shell,gas_station,36.242914,-86.736170,census_tract_10903
...,...,...,...,...,...
113,Dollar Tree,grocery_or_supermarket,36.245239,-86.762039,census_tract_10903
114,Merchants Metals,point_of_interest,36.254805,-86.758451,census_tract_10903
115,Staley Bus Sales / Staley Coach,car_dealer,36.268733,-86.754563,census_tract_10903
116,Discount Mufflers & Custom Exhaust,car_repair,36.254335,-86.758066,census_tract_10903


In [167]:
location_10904 = '36.23431, -86.77321'
radius_10904 = '2203'

In [344]:
ct10904 = []
ct10904_list = []
latitude10904 = []
longitude10904 = []
name10904 = []
type10904 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct10904_res = search_places_by_coordinate(location_10904, radius_10904, i, api_key) # running the API function through each type
    ct10904.append(ct10904_res) # creating a lists of the lists of each type of business
    for list in ct10904:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude10904.append(element['geometry']['location']['lat'])
            longitude10904.append(element['geometry']['location']['lng'])
            name10904.append(element['name'])
            type10904.append(element['types'][0])

In [345]:
name10904

['MAPCO',
 'Citgo',
 'MAPCO Mart',
 'Murphy Express',
 "TERRY'S",
 'Little Barn Market',
 'Exxon',
 'Dickerson Road Citgo',
 'Saint George Market',
 'G dragon mart',
 'MAPCO',
 'Citgo',
 'MAPCO Mart',
 'Murphy Express',
 "TERRY'S",
 'Little Barn Market',
 'Exxon',
 'Dickerson Road Citgo',
 'Saint George Market',
 'G dragon mart',
 'H G Hill Food Stores',
 'Family Dollar',
 'MAPCO',
 'Citgo',
 'MAPCO Mart',
 'Murphy Express',
 "TERRY'S",
 'Little Barn Market',
 'Exxon',
 'Dickerson Road Citgo',
 'Saint George Market',
 'G dragon mart',
 'H G Hill Food Stores',
 'Family Dollar',
 'TriStar Skyline Medical Center',
 'Skyline medical centrr',
 'MAPCO',
 'Citgo',
 'MAPCO Mart',
 'Murphy Express',
 "TERRY'S",
 'Little Barn Market',
 'Exxon',
 'Dickerson Road Citgo',
 'Saint George Market',
 'G dragon mart',
 'H G Hill Food Stores',
 'Family Dollar',
 'TriStar Skyline Medical Center',
 'Skyline medical centrr',
 'Walmart Supercenter',
 'Walgreens',
 'Xclusive Make-Up/SkinCare Studio',
 'Rainbo

In [347]:
df10904 = pd.DataFrame(zip(name10904, type10904, latitude10904, longitude10904),
               columns =['name', 'type', 'latitude', 'longitude'])

In [348]:
df10904.drop_duplicates(inplace=True, ignore_index=True)
df10904

,name,type,latitude,longitude
0,MAPCO,gas_station,36.229043,-86.758997
1,Citgo,gas_station,36.232554,-86.781664
2,MAPCO Mart,gas_station,36.218611,-86.762394
3,Murphy Express,gas_station,36.243486,-86.757311
4,TERRY'S,convenience_store,36.232562,-86.781676
...,...,...,...,...
98,Dollar Tree,grocery_or_supermarket,36.245239,-86.762039
99,Firehouse Discount Liquors,liquor_store,36.228929,-86.758036
100,Core & Main,point_of_interest,36.220334,-86.784996
101,Walmart Photo Center,electronics_store,36.244357,-86.762901


In [349]:
df10904['tract'] = 'census_tract_10904'
df10904

,name,type,latitude,longitude,tract
0,MAPCO,gas_station,36.229043,-86.758997,census_tract_10904
1,Citgo,gas_station,36.232554,-86.781664,census_tract_10904
2,MAPCO Mart,gas_station,36.218611,-86.762394,census_tract_10904
3,Murphy Express,gas_station,36.243486,-86.757311,census_tract_10904
4,TERRY'S,convenience_store,36.232562,-86.781676,census_tract_10904
...,...,...,...,...,...
98,Dollar Tree,grocery_or_supermarket,36.245239,-86.762039,census_tract_10904
99,Firehouse Discount Liquors,liquor_store,36.228929,-86.758036,census_tract_10904
100,Core & Main,point_of_interest,36.220334,-86.784996,census_tract_10904
101,Walmart Photo Center,electronics_store,36.244357,-86.762901,census_tract_10904


In [350]:
location_11001 = '36.22296, -86.75982'
radius_11001 = '2701'

In [351]:
ct11001 = []
ct11001_list = []
latitude11001 = []
longitude11001 = []
name11001 = []
type11001 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct11001_res = search_places_by_coordinate(location_11001, radius_11001, i, api_key) # running the API function through each type
    ct11001.append(ct11001_res) # creating a lists of the lists of each type of business
    for list in ct11001:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude11001.append(element['geometry']['location']['lat'])
            longitude11001.append(element['geometry']['location']['lng'])
            name11001.append(element['name'])
            type11001.append(element['types'][0])

In [352]:
df11001 = pd.DataFrame(zip(name11001, type11001, latitude11001, longitude11001),
               columns =['name', 'type', 'latitude', 'longitude'])

In [353]:
df11001.drop_duplicates(inplace=True, ignore_index=True)
df11001

,name,type,latitude,longitude
0,MAPCO Mart,gas_station,36.218611,-86.762394
1,MAPCO,gas_station,36.229043,-86.758997
2,Citgo,gas_station,36.232554,-86.781664
3,Shell,gas_station,36.208675,-86.778131
4,Murphy Express,gas_station,36.243486,-86.757311
...,...,...,...,...
137,Quad/Graphics Inc,point_of_interest,36.225156,-86.778245
138,Smart Buy Auto Sales,car_dealer,36.231355,-86.782061
139,Metro by T-Mobile,store,36.231120,-86.781180
140,Smartbox Moving and Storage,moving_company,36.205529,-86.770324


In [354]:
df11001['tract'] = 'census_tract_11001'
df11001

,name,type,latitude,longitude,tract
0,MAPCO Mart,gas_station,36.218611,-86.762394,census_tract_11001
1,MAPCO,gas_station,36.229043,-86.758997,census_tract_11001
2,Citgo,gas_station,36.232554,-86.781664,census_tract_11001
3,Shell,gas_station,36.208675,-86.778131,census_tract_11001
4,Murphy Express,gas_station,36.243486,-86.757311,census_tract_11001
...,...,...,...,...,...
137,Quad/Graphics Inc,point_of_interest,36.225156,-86.778245,census_tract_11001
138,Smart Buy Auto Sales,car_dealer,36.231355,-86.782061,census_tract_11001
139,Metro by T-Mobile,store,36.231120,-86.781180,census_tract_11001
140,Smartbox Moving and Storage,moving_company,36.205529,-86.770324,census_tract_11001


In [356]:
location_114 = '36.19852, -86.73378'
radius_114 = '1998'

In [357]:
ct114 = []
ct114_list = []
latitude114 = []
longitude114 = []
name114 = []
type114 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct114_res = search_places_by_coordinate(location_114, radius_114, i, api_key) # running the API function through each type
    ct114.append(ct114_res) # creating a lists of the lists of each type of business
    for list in ct114:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude114.append(element['geometry']['location']['lat'])
            longitude114.append(element['geometry']['location']['lng'])
            name114.append(element['name'])
            type114.append(element['types'][0])

In [359]:
df114 = pd.DataFrame(zip(name114, type114, latitude114, longitude114),
               columns =['name', 'type', 'latitude', 'longitude'])

In [360]:
df114.drop_duplicates(inplace=True, ignore_index=True)
df114

,name,type,latitude,longitude
0,Kroger Fuel Center,gas_station,36.210014,-86.733138
1,Kwik Sak,convenience_store,36.214093,-86.730432
2,Kroger Fuel Center,gas_station,36.185340,-86.747265
3,Speedway,gas_station,36.196900,-86.741704
4,Citgo,gas_station,36.205030,-86.724112
...,...,...,...,...
151,Redbox,movie_rental,36.211216,-86.732319
152,Dollar Tree,grocery_or_supermarket,36.211350,-86.733568
153,Spot's Pet Supply & Dog Wash,pet_store,36.189024,-86.745681
154,Look East,health,36.188998,-86.745750


In [361]:
df114['tract'] = 'census_tract_114'
df114

,name,type,latitude,longitude,tract
0,Kroger Fuel Center,gas_station,36.210014,-86.733138,census_tract_114
1,Kwik Sak,convenience_store,36.214093,-86.730432,census_tract_114
2,Kroger Fuel Center,gas_station,36.185340,-86.747265,census_tract_114
3,Speedway,gas_station,36.196900,-86.741704,census_tract_114
4,Citgo,gas_station,36.205030,-86.724112,census_tract_114
...,...,...,...,...,...
151,Redbox,movie_rental,36.211216,-86.732319,census_tract_114
152,Dollar Tree,grocery_or_supermarket,36.211350,-86.733568,census_tract_114
153,Spot's Pet Supply & Dog Wash,pet_store,36.189024,-86.745681,census_tract_114
154,Look East,health,36.188998,-86.745750,census_tract_114


In [362]:
location_118 = '36.18601, -86.76437'
radius_118 = '1818'

In [363]:
ct118 = []
ct118_list = []
latitude118 = []
longitude118= []
name118 = []
type118 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct118_res = search_places_by_coordinate(location_118, radius_118, i, api_key) # running the API function through each type
    ct118.append(ct118_res) # creating a lists of the lists of each type of business
    for list in ct118:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude118.append(element['geometry']['location']['lat'])
            longitude118.append(element['geometry']['location']['lng'])
            name118.append(element['name'])
            type118.append(element['types'][0])

In [364]:
df118 = pd.DataFrame(zip(name118, type118, latitude118, longitude118),
               columns =['name', 'type', 'latitude', 'longitude'])

In [365]:
df118.drop_duplicates(inplace=True, ignore_index=True)
df118

,name,type,latitude,longitude
0,Trillium,gas_station,36.178444,-86.774921
1,Kroger Fuel Center,gas_station,36.185340,-86.747265
2,Express Food Mart,convenience_store,36.177783,-86.749592
3,Marathon,gas_station,36.177779,-86.749787
4,TA Travel Center,gas_station,36.171075,-86.771986
...,...,...,...,...
187,Firestone,home_goods_store,36.181067,-86.749957
188,The Turnip Truck - East Nashville,grocery_or_supermarket,36.173825,-86.760016
189,FASTSIGNS,point_of_interest,36.182740,-86.749100
190,Mutts + Meows Natural Pet Market,point_of_interest,36.176283,-86.754036


In [366]:
df118['tract'] = 'census_tract_118'
df118

,name,type,latitude,longitude,tract
0,Trillium,gas_station,36.178444,-86.774921,census_tract_118
1,Kroger Fuel Center,gas_station,36.185340,-86.747265,census_tract_118
2,Express Food Mart,convenience_store,36.177783,-86.749592,census_tract_118
3,Marathon,gas_station,36.177779,-86.749787,census_tract_118
4,TA Travel Center,gas_station,36.171075,-86.771986,census_tract_118
...,...,...,...,...,...
187,Firestone,home_goods_store,36.181067,-86.749957,census_tract_118
188,The Turnip Truck - East Nashville,grocery_or_supermarket,36.173825,-86.760016,census_tract_118
189,FASTSIGNS,point_of_interest,36.182740,-86.749100,census_tract_118
190,Mutts + Meows Natural Pet Market,point_of_interest,36.176283,-86.754036,census_tract_118


In [368]:
location_119= '36.17908, -86.75906'
radius_119= '1658'

In [369]:
ct119 = []
ct119_list = []
latitude119 = []
longitude119= []
name119 = []
type119 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct119_res = search_places_by_coordinate(location_119, radius_119, i, api_key) # running the API function through each type
    ct119.append(ct119_res) # creating a lists of the lists of each type of business
    for list in ct119:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude119.append(element['geometry']['location']['lat'])
            longitude119.append(element['geometry']['location']['lng'])
            name119.append(element['name'])
            type119.append(element['types'][0])

In [370]:
df119 = pd.DataFrame(zip(name119, type119, latitude119, longitude119),
               columns =['name', 'type', 'latitude', 'longitude'])

In [371]:
df119.drop_duplicates(inplace=True, ignore_index=True)
df119

,name,type,latitude,longitude
0,TA Travel Center,gas_station,36.171075,-86.771986
1,Express Food Mart,convenience_store,36.177783,-86.749592
2,Marathon,gas_station,36.177779,-86.749787
3,Shell,gas_station,36.171552,-86.751873
4,Kroger Fuel Center,gas_station,36.185340,-86.747265
...,...,...,...,...
180,Pearl Snap Studios,electronics_store,36.177521,-86.748835
181,Vintage King Nashville,electronics_store,36.182924,-86.749573
182,FASTSIGNS,point_of_interest,36.182740,-86.749100
183,Board & Brush Creative Studio - Nashville,home_goods_store,36.186987,-86.764588


In [372]:
df119['tract'] = 'census_tract_119'
df119

,name,type,latitude,longitude,tract
0,TA Travel Center,gas_station,36.171075,-86.771986,census_tract_119
1,Express Food Mart,convenience_store,36.177783,-86.749592,census_tract_119
2,Marathon,gas_station,36.177779,-86.749787,census_tract_119
3,Shell,gas_station,36.171552,-86.751873,census_tract_119
4,Kroger Fuel Center,gas_station,36.185340,-86.747265,census_tract_119
...,...,...,...,...,...
180,Pearl Snap Studios,electronics_store,36.177521,-86.748835,census_tract_119
181,Vintage King Nashville,electronics_store,36.182924,-86.749573,census_tract_119
182,FASTSIGNS,point_of_interest,36.182740,-86.749100,census_tract_119
183,Board & Brush Creative Studio - Nashville,home_goods_store,36.186987,-86.764588,census_tract_119


In [373]:
location_126= '36.18710, -86.77198'
radius_126= '1672'

In [374]:
ct126 = []
ct126_list = []
latitude126 = []
longitude126= []
name126 = []
type126 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct126_res = search_places_by_coordinate(location_126, radius_126, i, api_key) # running the API function through each type
    ct126.append(ct126_res) # creating a lists of the lists of each type of business
    for list in ct126:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude126.append(element['geometry']['location']['lat'])
            longitude126.append(element['geometry']['location']['lng'])
            name126.append(element['name'])
            type126.append(element['types'][0])

In [375]:
df126 = pd.DataFrame(zip(name126, type126, latitude126, longitude126),
               columns =['name', 'type', 'latitude', 'longitude'])

In [376]:
df126.drop_duplicates(inplace=True, ignore_index=True)
df126

,name,type,latitude,longitude
0,Trillium,gas_station,36.178444,-86.774921
1,Z Mart,gas_station,36.184732,-86.773392
2,Citgo,gas_station,36.183447,-86.773557
3,Safelite-Nashville,gas_station,36.188180,-86.777185
4,MyStik,gas_station,36.176671,-86.774879
...,...,...,...,...
115,Elite Multimedia Productions,electronics_store,36.183976,-86.776995
116,"Phoenix Foods, Inc.",grocery_or_supermarket,36.184491,-86.776060
117,Hair World Beauty Supply,point_of_interest,36.187112,-86.772894
118,SNT Market,convenience_store,36.186622,-86.772969


In [377]:
df126['tract'] = 'census_tract_126'
df126

,name,type,latitude,longitude,tract
0,Trillium,gas_station,36.178444,-86.774921,census_tract_126
1,Z Mart,gas_station,36.184732,-86.773392,census_tract_126
2,Citgo,gas_station,36.183447,-86.773557,census_tract_126
3,Safelite-Nashville,gas_station,36.188180,-86.777185,census_tract_126
4,MyStik,gas_station,36.176671,-86.774879,census_tract_126
...,...,...,...,...,...
115,Elite Multimedia Productions,electronics_store,36.183976,-86.776995,census_tract_126
116,"Phoenix Foods, Inc.",grocery_or_supermarket,36.184491,-86.776060,census_tract_126
117,Hair World Beauty Supply,point_of_interest,36.187112,-86.772894,census_tract_126
118,SNT Market,convenience_store,36.186622,-86.772969,census_tract_126


In [378]:
location_12701= '36.22121, -86.79463'
radius_12701= '2946'

In [379]:
ct12701 = []
ct12701_list = []
latitude12701 = []
longitude12701 = []
name12701 = []
type12701 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct12701_res = search_places_by_coordinate(location_12701, radius_12701, i, api_key) # running the API function through each type
    ct12701.append(ct12701_res) # creating a lists of the lists of each type of business
    for list in ct12701:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude12701.append(element['geometry']['location']['lat'])
            longitude12701.append(element['geometry']['location']['lng'])
            name12701.append(element['name'])
            type12701.append(element['types'][0])

In [380]:
df12701 = pd.DataFrame(zip(name12701, type12701, latitude12701, longitude12701),
               columns =['name', 'type', 'latitude', 'longitude'])

In [381]:
df12701.drop_duplicates(inplace=True, ignore_index=True)
df12701

,name,type,latitude,longitude
0,Citgo,gas_station,36.232554,-86.781664
1,Shell,gas_station,36.208675,-86.778131
2,MAPCO Mart,gas_station,36.218611,-86.762394
3,TERRY'S,convenience_store,36.232562,-86.781676
4,Tiger Mart,convenience_store,36.207392,-86.778339
...,...,...,...,...
91,FLWR Shop,florist,36.218127,-86.762181
92,Empire Today,home_goods_store,36.223656,-86.782890
93,Talking with Dr. Toy,point_of_interest,36.200386,-86.811482
94,Pinnacle Colors,point_of_interest,36.196268,-86.793489


In [382]:
df12701['tract'] = 'census_tract_12701'
df12701

,name,type,latitude,longitude,tract
0,Citgo,gas_station,36.232554,-86.781664,census_tract_12701
1,Shell,gas_station,36.208675,-86.778131,census_tract_12701
2,MAPCO Mart,gas_station,36.218611,-86.762394,census_tract_12701
3,TERRY'S,convenience_store,36.232562,-86.781676,census_tract_12701
4,Tiger Mart,convenience_store,36.207392,-86.778339,census_tract_12701
...,...,...,...,...,...
91,FLWR Shop,florist,36.218127,-86.762181,census_tract_12701
92,Empire Today,home_goods_store,36.223656,-86.782890,census_tract_12701
93,Talking with Dr. Toy,point_of_interest,36.200386,-86.811482,census_tract_12701
94,Pinnacle Colors,point_of_interest,36.196268,-86.793489,census_tract_12701


In [383]:
location_12702= '36.20596, -86.79612'
radius_12702= '2824'

In [384]:
ct12702 = []
ct12702_list = []
latitude12702 = []
longitude12702 = []
name12702 = []
type12702 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct12702_res = search_places_by_coordinate(location_12702, radius_12702, i, api_key) # running the API function through each type
    ct12702.append(ct12702_res) # creating a lists of the lists of each type of business
    for list in ct12702:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude12702.append(element['geometry']['location']['lat'])
            longitude12702.append(element['geometry']['location']['lng'])
            name12702.append(element['name'])
            type12702.append(element['types'][0])

In [385]:
df12702 = pd.DataFrame(zip(name12702, type12702, latitude12702, longitude12702),
               columns =['name', 'type', 'latitude', 'longitude'])

In [386]:
df12702.drop_duplicates(inplace=True, ignore_index=True)
df12702

,name,type,latitude,longitude
0,Exxon,gas_station,36.188416,-86.798098
1,Shell,gas_station,36.208675,-86.778131
2,Mobil,gas_station,36.188316,-86.798082
3,Tiger Mart,convenience_store,36.188453,-86.798177
4,Tiger Mart,convenience_store,36.207392,-86.778339
...,...,...,...,...
130,Bright Event Productions,electronics_store,36.227140,-86.786553
131,4Wall Nashville,electronics_store,36.187525,-86.779824
132,Oldham Chemical Co,home_goods_store,36.192426,-86.772898
133,Car Source Auto Sales,car_dealer,36.207268,-86.767494


In [387]:
df12702['tract'] = 'census_tract_12702'
df12702

,name,type,latitude,longitude,tract
0,Exxon,gas_station,36.188416,-86.798098,census_tract_12702
1,Shell,gas_station,36.208675,-86.778131,census_tract_12702
2,Mobil,gas_station,36.188316,-86.798082,census_tract_12702
3,Tiger Mart,convenience_store,36.188453,-86.798177,census_tract_12702
4,Tiger Mart,convenience_store,36.207392,-86.778339,census_tract_12702
...,...,...,...,...,...
130,Bright Event Productions,electronics_store,36.227140,-86.786553,census_tract_12702
131,4Wall Nashville,electronics_store,36.187525,-86.779824,census_tract_12702
132,Oldham Chemical Co,home_goods_store,36.192426,-86.772898,census_tract_12702
133,Car Source Auto Sales,car_dealer,36.207268,-86.767494,census_tract_12702


In [388]:
location_12801= '36.20801, -86.83523'
radius_12801= '2915'

In [389]:
ct12801 = []
ct12801_list = []
latitude12801 = []
longitude12801 = []
name12801 = []
type12801 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct12801_res = search_places_by_coordinate(location_12801, radius_12801, i, api_key) # running the API function through each type
    ct12801.append(ct12801_res) # creating a lists of the lists of each type of business
    for list in ct12801:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude12801.append(element['geometry']['location']['lat'])
            longitude12801.append(element['geometry']['location']['lng'])
            name12801.append(element['name'])
            type12801.append(element['types'][0])

In [390]:
df12801 = pd.DataFrame(zip(name12801, type12801, latitude12801, longitude12801),
               columns =['name', 'type', 'latitude', 'longitude'])

In [391]:
df12801.drop_duplicates(inplace=True, ignore_index=True)
df12801

,name,type,latitude,longitude
0,Kroger Fuel Center,gas_station,36.210674,-86.838564
1,Little Barn Market,gas_station,36.209159,-86.839701
2,F & B Mart,gas_station,36.207746,-86.839515
3,Citigo,gas_station,36.208055,-86.839666
4,Bordeaux Grab&go,gas_station,36.209226,-86.839838
...,...,...,...,...
97,Sunfish import auto sales,car_dealer,36.202792,-86.839995
98,Parrish Equipment Service,home_goods_store,36.202614,-86.839819
99,Lulu's Discount Tobacco,point_of_interest,36.198559,-86.839423
100,Hair Global beauty supply,point_of_interest,36.198538,-86.839510


In [392]:
df12801['tract'] = 'census_tract_12801'
df12801

,name,type,latitude,longitude,tract
0,Kroger Fuel Center,gas_station,36.210674,-86.838564,census_tract_12801
1,Little Barn Market,gas_station,36.209159,-86.839701,census_tract_12801
2,F & B Mart,gas_station,36.207746,-86.839515,census_tract_12801
3,Citigo,gas_station,36.208055,-86.839666,census_tract_12801
4,Bordeaux Grab&go,gas_station,36.209226,-86.839838,census_tract_12801
...,...,...,...,...,...
97,Sunfish import auto sales,car_dealer,36.202792,-86.839995,census_tract_12801
98,Parrish Equipment Service,home_goods_store,36.202614,-86.839819,census_tract_12801
99,Lulu's Discount Tobacco,point_of_interest,36.198559,-86.839423,census_tract_12801
100,Hair Global beauty supply,point_of_interest,36.198538,-86.839510,census_tract_12801


In [393]:
location_12802= '36.19057, -86.85533'
radius_12802= '3509'

In [394]:
ct12802 = []
ct12802_list = []
latitude12802 = []
longitude12802 = []
name12802 = []
type12802 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct12802_res = search_places_by_coordinate(location_12802, radius_12802, i, api_key) # running the API function through each type
    ct12802.append(ct12802_res) # creating a lists of the lists of each type of business
    for list in ct12802:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude12802.append(element['geometry']['location']['lat'])
            longitude12802.append(element['geometry']['location']['lng'])
            name12802.append(element['name'])
            type12802.append(element['types'][0])

In [395]:
df12802 = pd.DataFrame(zip(name12802, type12802, latitude12802, longitude12802),
               columns =['name', 'type', 'latitude', 'longitude'])

In [396]:
df12802.drop_duplicates(inplace=True, ignore_index=True)
df12802

,name,type,latitude,longitude
0,Pilot Travel Center,gas_station,36.172260,-86.865304
1,Magellan Midstream Partners LP,gas_station,36.169289,-86.863084
2,Kroger Fuel Center,gas_station,36.210674,-86.838564
3,Marathon Petroleum 51st,gas_station,36.165596,-86.849273
4,Little Barn Market,gas_station,36.209159,-86.839701
...,...,...,...,...
130,AutoZone Auto Parts,car_repair,36.198952,-86.837788
131,Allegra Marketing Print Mail,point_of_interest,36.161549,-86.857106
132,Rent-A-Center,furniture_store,36.198958,-86.839645
133,B & B Used Appliances,home_goods_store,36.200785,-86.838939


In [397]:
df12802['tract'] = 'census_tract_12802'
df12802

,name,type,latitude,longitude,tract
0,Pilot Travel Center,gas_station,36.172260,-86.865304,census_tract_12802
1,Magellan Midstream Partners LP,gas_station,36.169289,-86.863084,census_tract_12802
2,Kroger Fuel Center,gas_station,36.210674,-86.838564,census_tract_12802
3,Marathon Petroleum 51st,gas_station,36.165596,-86.849273,census_tract_12802
4,Little Barn Market,gas_station,36.209159,-86.839701,census_tract_12802
...,...,...,...,...,...
130,AutoZone Auto Parts,car_repair,36.198952,-86.837788,census_tract_12802
131,Allegra Marketing Print Mail,point_of_interest,36.161549,-86.857106,census_tract_12802
132,Rent-A-Center,furniture_store,36.198958,-86.839645,census_tract_12802
133,B & B Used Appliances,home_goods_store,36.200785,-86.838939,census_tract_12802


In [398]:
location_13601= '36.16390, -86.82741'
radius_13601= '2448'

In [399]:
ct13601 = []
ct13601_list = []
latitude13601 = []
longitude13601 = []
name13601 = []
type13601 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct13601_res = search_places_by_coordinate(location_13601, radius_13601, i, api_key) # running the API function through each type
    ct13601.append(ct13601_res) # creating a lists of the lists of each type of business
    for list in ct13601:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude13601.append(element['geometry']['location']['lat'])
            longitude13601.append(element['geometry']['location']['lng'])
            name13601.append(element['name'])
            type13601.append(element['types'][0])

In [400]:
df13601 = pd.DataFrame(zip(name13601, type13601, latitude13601, longitude13601),
               columns =['name', 'type', 'latitude', 'longitude'])

In [401]:
df13601.drop_duplicates(inplace=True, ignore_index=True)
df13601

,name,type,latitude,longitude
0,Petroleum Equipment Co Inc,gas_station,36.153636,-86.835094
1,Shell,gas_station,36.152982,-86.842490
2,Marathon Petroleum 51st,gas_station,36.165596,-86.849273
3,Shell,gas_station,36.156657,-86.806903
4,MAPCO,gas_station,36.152134,-86.838028
...,...,...,...,...
212,The Powder Room Mobile,point_of_interest,36.169007,-86.807189
213,On Promos,point_of_interest,36.157516,-86.847704
214,51st Deli,food,36.161907,-86.848870
215,Dollar General,drugstore,36.163044,-86.849561


In [402]:
df13601['tract'] = 'census_tract_13601'
df13601

,name,type,latitude,longitude,tract
0,Petroleum Equipment Co Inc,gas_station,36.153636,-86.835094,census_tract_13601
1,Shell,gas_station,36.152982,-86.842490,census_tract_13601
2,Marathon Petroleum 51st,gas_station,36.165596,-86.849273,census_tract_13601
3,Shell,gas_station,36.156657,-86.806903,census_tract_13601
4,MAPCO,gas_station,36.152134,-86.838028,census_tract_13601
...,...,...,...,...,...
212,The Powder Room Mobile,point_of_interest,36.169007,-86.807189,census_tract_13601
213,On Promos,point_of_interest,36.157516,-86.847704,census_tract_13601
214,51st Deli,food,36.161907,-86.848870,census_tract_13601
215,Dollar General,drugstore,36.163044,-86.849561,census_tract_13601


In [403]:
location_13602= '36.17144, -86.82851'
radius_13602= '1885'

In [404]:
ct13602 = []
ct13602_list = []
latitude13602 = []
longitude13602 = []
name13602 = []
type13602 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct13602_res = search_places_by_coordinate(location_13602, radius_13602, i, api_key) # running the API function through each type
    ct13602.append(ct13602_res) # creating a lists of the lists of each type of business
    for list in ct13602:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude13602.append(element['geometry']['location']['lat'])
            longitude13602.append(element['geometry']['location']['lng'])
            name13602.append(element['name'])
            type13602.append(element['types'][0])

In [405]:
df13602 = pd.DataFrame(zip(name13602, type13602, latitude13602, longitude13602),
               columns =['name', 'type', 'latitude', 'longitude'])

In [406]:
df13602.drop_duplicates(inplace=True, ignore_index=True)
df13602

,name,type,latitude,longitude
0,Marathon Gas,gas_station,36.167786,-86.821652
1,Marathon Gas,gas_station,36.167704,-86.820956
2,Pacific Pride,gas_station,36.179149,-86.824220
3,A header Lane Church of Christ,gas_station,36.171479,-86.816625
4,Schrader. Lane church,gas_station,36.171479,-86.816625
...,...,...,...,...
85,Premium Cigars and Spirits,store,36.169047,-86.808049
86,FARIS TIRE SHOP,car_repair,36.181502,-86.811954
87,Discounts Mart,convenience_store,36.179696,-86.810467
88,Northwest Liquors,liquor_store,36.179696,-86.810467


In [407]:
df13602['tract'] = 'census_tract_13602'
df13602

,name,type,latitude,longitude,tract
0,Marathon Gas,gas_station,36.167786,-86.821652,census_tract_13602
1,Marathon Gas,gas_station,36.167704,-86.820956,census_tract_13602
2,Pacific Pride,gas_station,36.179149,-86.824220,census_tract_13602
3,A header Lane Church of Christ,gas_station,36.171479,-86.816625,census_tract_13602
4,Schrader. Lane church,gas_station,36.171479,-86.816625,census_tract_13602
...,...,...,...,...,...
85,Premium Cigars and Spirits,store,36.169047,-86.808049,census_tract_13602
86,FARIS TIRE SHOP,car_repair,36.181502,-86.811954,census_tract_13602
87,Discounts Mart,convenience_store,36.179696,-86.810467,census_tract_13602
88,Northwest Liquors,liquor_store,36.179696,-86.810467,census_tract_13602


In [408]:
location_137= '36.19115, -86.80853'
radius_137= '2551'

In [409]:
ct137 = []
ct137_list = []
latitude137 = []
longitude137 = []
name137 = []
type137 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct137_res = search_places_by_coordinate(location_137, radius_137, i, api_key) # running the API function through each type
    ct137.append(ct137_res) # creating a lists of the lists of each type of business
    for list in ct137:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude137.append(element['geometry']['location']['lat'])
            longitude137.append(element['geometry']['location']['lng'])
            name137.append(element['name'])
            type137.append(element['types'][0])

In [412]:
df137 = pd.DataFrame(zip(name137, type137, latitude137, longitude137),
               columns =['name', 'type', 'latitude', 'longitude'])

In [413]:
df137.drop_duplicates(inplace=True, ignore_index=True)
df137

,name,type,latitude,longitude
0,Exxon,gas_station,36.188416,-86.798098
1,Mobil,gas_station,36.188316,-86.798082
2,Kroger Fuel Center,gas_station,36.175368,-86.793211
3,Tiger Mart,convenience_store,36.188453,-86.798177
4,bp,gas_station,36.172959,-86.791613
...,...,...,...,...
168,Kroger Bakery,bakery,36.175818,-86.794357
169,Bordeaux Food,grocery_or_supermarket,36.193048,-86.832462
170,Kroger Deli,food,36.175858,-86.793877
171,Cinematech Custom Home Integration,electronics_store,36.179983,-86.788290


In [414]:
df137['tract'] = 'census_tract_137'
df137

,name,type,latitude,longitude,tract
0,Exxon,gas_station,36.188416,-86.798098,census_tract_137
1,Mobil,gas_station,36.188316,-86.798082,census_tract_137
2,Kroger Fuel Center,gas_station,36.175368,-86.793211,census_tract_137
3,Tiger Mart,convenience_store,36.188453,-86.798177,census_tract_137
4,bp,gas_station,36.172959,-86.791613,census_tract_137
...,...,...,...,...,...
168,Kroger Bakery,bakery,36.175818,-86.794357,census_tract_137
169,Bordeaux Food,grocery_or_supermarket,36.193048,-86.832462,census_tract_137
170,Kroger Deli,food,36.175858,-86.793877,census_tract_137
171,Cinematech Custom Home Integration,electronics_store,36.179983,-86.788290,census_tract_137


In [415]:
location_138= '36.17565, -86.81480'
radius_138= '1559'

In [416]:
ct138 = []
ct138_list = []
latitude138 = []
longitude138 = []
name138 = []
type138 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct138_res = search_places_by_coordinate(location_138, radius_138, i, api_key) # running the API function through each type
    ct138.append(ct138_res) # creating a lists of the lists of each type of business
    for list in ct138:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude138.append(element['geometry']['location']['lat'])
            longitude138.append(element['geometry']['location']['lng'])
            name138.append(element['name'])
            type138.append(element['types'][0])

In [417]:
df138 = pd.DataFrame(zip(name138, type138, latitude138, longitude138),
               columns =['name', 'type', 'latitude', 'longitude'])

In [419]:
df138.drop_duplicates(inplace=True, ignore_index=True)
df138

,name,type,latitude,longitude
0,Pauls Market,gas_station,36.169698,-86.807180
1,A header Lane Church of Christ,gas_station,36.171479,-86.816625
2,Schrader. Lane church,gas_station,36.171479,-86.816625
3,Pacific Pride,gas_station,36.179149,-86.824220
4,Kings Market,gas_station,36.169579,-86.807004
...,...,...,...,...
113,Carter's Family Florist,florist,36.182154,-86.819056
114,Inner Eaze,home_goods_store,36.168536,-86.811952
115,"Tri Star Transport, LLC",point_of_interest,36.178302,-86.823849
116,SAM'S MARKET,grocery_or_supermarket,36.183237,-86.812551


In [420]:
df138['tract'] = 'census_tract_138'
df138

,name,type,latitude,longitude,tract
0,Pauls Market,gas_station,36.169698,-86.807180,census_tract_138
1,A header Lane Church of Christ,gas_station,36.171479,-86.816625,census_tract_138
2,Schrader. Lane church,gas_station,36.171479,-86.816625,census_tract_138
3,Pacific Pride,gas_station,36.179149,-86.824220,census_tract_138
4,Kings Market,gas_station,36.169579,-86.807004,census_tract_138
...,...,...,...,...,...
113,Carter's Family Florist,florist,36.182154,-86.819056,census_tract_138
114,Inner Eaze,home_goods_store,36.168536,-86.811952,census_tract_138
115,"Tri Star Transport, LLC",point_of_interest,36.178302,-86.823849,census_tract_138
116,SAM'S MARKET,grocery_or_supermarket,36.183237,-86.812551,census_tract_138


In [421]:
location_139= '36.17571, -86.80562'
radius_139= '1514'

In [422]:
ct139 = []
ct139_list = []
latitude139 = []
longitude139 = []
name139 = []
type139 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct139_res = search_places_by_coordinate(location_139, radius_139, i, api_key) # running the API function through each type
    ct139.append(ct139_res) # creating a lists of the lists of each type of business
    for list in ct139:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude139.append(element['geometry']['location']['lat'])
            longitude139.append(element['geometry']['location']['lng'])
            name139.append(element['name'])
            type139.append(element['types'][0])

In [423]:
df139 = pd.DataFrame(zip(name139, type139, latitude139, longitude139),
               columns =['name', 'type', 'latitude', 'longitude'])

In [424]:
df139.drop_duplicates(inplace=True, ignore_index=True)
df139

,name,type,latitude,longitude
0,Kroger Fuel Center,gas_station,36.175368,-86.793211
1,bp,gas_station,36.172959,-86.791613
2,Pauls Market,gas_station,36.169698,-86.807180
3,Kings Market,gas_station,36.169579,-86.807004
4,A header Lane Church of Christ,gas_station,36.171479,-86.816625
...,...,...,...,...
132,Provision,food,36.163761,-86.797701
133,Lorraine Frances,jewelry_store,36.164952,-86.795403
134,Wireless Z,point_of_interest,36.180924,-86.811705
135,Nelson's Green Brier Distillery,tourist_attraction,36.163509,-86.798218


In [425]:
df139['tract'] = 'census_tract_139'
df139

,name,type,latitude,longitude,tract
0,Kroger Fuel Center,gas_station,36.175368,-86.793211,census_tract_139
1,bp,gas_station,36.172959,-86.791613,census_tract_139
2,Pauls Market,gas_station,36.169698,-86.807180,census_tract_139
3,Kings Market,gas_station,36.169579,-86.807004,census_tract_139
4,A header Lane Church of Christ,gas_station,36.171479,-86.816625,census_tract_139
...,...,...,...,...,...
132,Provision,food,36.163761,-86.797701,census_tract_139
133,Lorraine Frances,jewelry_store,36.164952,-86.795403,census_tract_139
134,Wireless Z,point_of_interest,36.180924,-86.811705,census_tract_139
135,Nelson's Green Brier Distillery,tourist_attraction,36.163509,-86.798218,census_tract_139


In [426]:
location_142= '36.16693, -86.80365'
radius_142= '1640'

In [427]:
ct142 = []
ct142_list = []
latitude142 = []
longitude142 = []
name142 = []
type142 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct142_res = search_places_by_coordinate(location_142, radius_142, i, api_key) # running the API function through each type
    ct142.append(ct142_res) # creating a lists of the lists of each type of business
    for list in ct142:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude142.append(element['geometry']['location']['lat'])
            longitude142.append(element['geometry']['location']['lng'])
            name142.append(element['name'])
            type142.append(element['types'][0])

In [428]:
df142 = pd.DataFrame(zip(name142, type142, latitude142, longitude142),
               columns =['name', 'type', 'latitude', 'longitude'])

In [429]:
df142.drop_duplicates(inplace=True, ignore_index=True)
df142

,name,type,latitude,longitude
0,Shell,gas_station,36.156657,-86.806903
1,bp,gas_station,36.172959,-86.791613
2,Kroger Fuel Center,gas_station,36.175368,-86.793211
3,Exxon,gas_station,36.160241,-86.793982
4,Pauls Market,gas_station,36.169698,-86.807180
...,...,...,...,...
223,Fords Herbal & Doula Services,health,36.170824,-86.788982
224,Rite Aid Pharmacy,pharmacy,36.173862,-86.790796
225,Green Fleet Bicycle Shop,bicycle_store,36.172717,-86.794401
226,Jubilee Flowers & Gifts,florist,36.170144,-86.804531


In [430]:
df142['tract'] = 'census_tract_142'
df142

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.156657,-86.806903,census_tract_142
1,bp,gas_station,36.172959,-86.791613,census_tract_142
2,Kroger Fuel Center,gas_station,36.175368,-86.793211,census_tract_142
3,Exxon,gas_station,36.160241,-86.793982,census_tract_142
4,Pauls Market,gas_station,36.169698,-86.807180,census_tract_142
...,...,...,...,...,...
223,Fords Herbal & Doula Services,health,36.170824,-86.788982,census_tract_142
224,Rite Aid Pharmacy,pharmacy,36.173862,-86.790796,census_tract_142
225,Green Fleet Bicycle Shop,bicycle_store,36.172717,-86.794401,census_tract_142
226,Jubilee Flowers & Gifts,florist,36.170144,-86.804531,census_tract_142


In [431]:
location_143= '36.16124, -86.81564'
radius_143= '1552'

In [432]:
ct143 = []
ct143_list = []
latitude143 = []
longitude143 = []
name143 = []
type143 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct143_res = search_places_by_coordinate(location_143, radius_143, i, api_key) # running the API function through each type
    ct143.append(ct143_res) # creating a lists of the lists of each type of business
    for list in ct143:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude143.append(element['geometry']['location']['lat'])
            longitude143.append(element['geometry']['location']['lng'])
            name143.append(element['name'])
            type143.append(element['types'][0])

In [433]:
df143 = pd.DataFrame(zip(name143, type143, latitude143, longitude143),
               columns =['name', 'type', 'latitude', 'longitude'])

In [434]:
df143.drop_duplicates(inplace=True, ignore_index=True)
df143

,name,type,latitude,longitude
0,Shell,gas_station,36.156657,-86.806903
1,Pauls Market,gas_station,36.169698,-86.807180
2,Marathon Gas,gas_station,36.167704,-86.820956
3,Marathon Gas,gas_station,36.167786,-86.821652
4,A header Lane Church of Christ,gas_station,36.171479,-86.816625
...,...,...,...,...
181,"James T. Sowell, DPH",pharmacy,36.153329,-86.809089
182,Syncor Pharmacy Services Center,pharmacy,36.148597,-86.808593
183,A Higher Standard,electronics_store,36.163668,-86.798734
184,Cliff's Cabinet Company,furniture_store,36.160168,-86.805333


In [435]:
df143['tract'] = 'census_tract_143'
df143

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.156657,-86.806903,census_tract_143
1,Pauls Market,gas_station,36.169698,-86.807180,census_tract_143
2,Marathon Gas,gas_station,36.167704,-86.820956,census_tract_143
3,Marathon Gas,gas_station,36.167786,-86.821652,census_tract_143
4,A header Lane Church of Christ,gas_station,36.171479,-86.816625,census_tract_143
...,...,...,...,...,...
181,"James T. Sowell, DPH",pharmacy,36.153329,-86.809089,census_tract_143
182,Syncor Pharmacy Services Center,pharmacy,36.148597,-86.808593,census_tract_143
183,A Higher Standard,electronics_store,36.163668,-86.798734,census_tract_143
184,Cliff's Cabinet Company,furniture_store,36.160168,-86.805333,census_tract_143


In [436]:
location_144= '36.15766, -86.80573'
radius_144= '2194'

In [437]:
ct144 = []
ct144_list = []
latitude144 = []
longitude144 = []
name144 = []
type144 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct144_res = search_places_by_coordinate(location_144, radius_144, i, api_key) # running the API function through each type
    ct144.append(ct144_res) # creating a lists of the lists of each type of business
    for list in ct144:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude144.append(element['geometry']['location']['lat'])
            longitude144.append(element['geometry']['location']['lng'])
            name144.append(element['name'])
            type144.append(element['types'][0])

In [441]:
df144 = pd.DataFrame(zip(name144, type144, latitude144, longitude144),
               columns =['name', 'type', 'latitude', 'longitude'])

In [442]:
df144.drop_duplicates(inplace=True, ignore_index=True)
df144

,name,type,latitude,longitude
0,Shell,gas_station,36.156657,-86.806903
1,Shell,gas_station,36.153856,-86.794865
2,Exxon,gas_station,36.160241,-86.793982
3,bp,gas_station,36.172959,-86.791613
4,Exxon,gas_station,36.156550,-86.786897
...,...,...,...,...
283,"RadioActive Entertainment, LLC",electronics_store,36.144432,-86.812194
284,Heads Up Hair Salon,beauty_salon,36.149034,-86.805971
285,Robin Haley Jewelry,jewelry_store,36.143975,-86.812816
286,Beaman Buick GMC,finance,36.154030,-86.792411


In [443]:
df144['tract'] = 'census_tract_144'
df144

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.156657,-86.806903,census_tract_144
1,Shell,gas_station,36.153856,-86.794865,census_tract_144
2,Exxon,gas_station,36.160241,-86.793982,census_tract_144
3,bp,gas_station,36.172959,-86.791613,census_tract_144
4,Exxon,gas_station,36.156550,-86.786897,census_tract_144
...,...,...,...,...,...
283,"RadioActive Entertainment, LLC",electronics_store,36.144432,-86.812194,census_tract_144
284,Heads Up Hair Salon,beauty_salon,36.149034,-86.805971,census_tract_144
285,Robin Haley Jewelry,jewelry_store,36.143975,-86.812816,census_tract_144
286,Beaman Buick GMC,finance,36.154030,-86.792411,census_tract_144


In [451]:
location_148= '36.15109, -86.75970'
radius_148= '1430'

In [452]:
ct148 = []
ct148_list = []
latitude148 = []
longitude148 = []
name148 = []
type148 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct148_res = search_places_by_coordinate(location_148, radius_148, i, api_key) # running the API function through each type
    ct148.append(ct148_res) # creating a lists of the lists of each type of business
    for list in ct148:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude148.append(element['geometry']['location']['lat'])
            longitude148.append(element['geometry']['location']['lng'])
            name148.append(element['name'])
            type148.append(element['types'][0])

In [453]:
df148 = pd.DataFrame(zip(name148, type148, latitude148, longitude148),
               columns =['name', 'type', 'latitude', 'longitude'])

In [454]:
df148.drop_duplicates(inplace=True, ignore_index=True)
df148

,name,type,latitude,longitude
0,Shell Island,gas_station,36.147766,-86.758112
1,GC MART,gas_station,36.147725,-86.758041
2,Ebt,gas_station,36.147338,-86.757676
3,Mucho Express,convenience_store,36.146841,-86.757527
4,Eagle Parking - Shell,gas_station,36.157081,-86.770247
...,...,...,...,...
141,Flying Wheels Auto,car_repair,36.140426,-86.758121
142,RNB Cycles,point_of_interest,36.147317,-86.772057
143,"Elite Auto Group And Services, Inc.",car_repair,36.144172,-86.749330
144,"Elite Auto Group and Services, INC",car_dealer,36.144171,-86.749268


In [455]:
df148['tract'] = 'census_tract_148'
df148

,name,type,latitude,longitude,tract
0,Shell Island,gas_station,36.147766,-86.758112,census_tract_148
1,GC MART,gas_station,36.147725,-86.758041,census_tract_148
2,Ebt,gas_station,36.147338,-86.757676,census_tract_148
3,Mucho Express,convenience_store,36.146841,-86.757527,census_tract_148
4,Eagle Parking - Shell,gas_station,36.157081,-86.770247,census_tract_148
...,...,...,...,...,...
141,Flying Wheels Auto,car_repair,36.140426,-86.758121,census_tract_148
142,RNB Cycles,point_of_interest,36.147317,-86.772057,census_tract_148
143,"Elite Auto Group And Services, Inc.",car_repair,36.144172,-86.749330,census_tract_148
144,"Elite Auto Group and Services, INC",car_dealer,36.144171,-86.749268,census_tract_148


In [450]:
location_15630= '36.03003, -86.61451'
radius_15630= '3149'

In [456]:
ct15630 = []
ct15630_list = []
latitude15630 = []
longitude15630 = []
name15630 = []
type15630 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct15630_res = search_places_by_coordinate(location_15630, radius_15630, i, api_key) # running the API function through each type
    ct15630.append(ct15630_res) # creating a lists of the lists of each type of business
    for list in ct15630:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude15630.append(element['geometry']['location']['lat'])
            longitude15630.append(element['geometry']['location']['lng'])
            name15630.append(element['name'])
            type15630.append(element['types'][0])

In [457]:
df15630 = pd.DataFrame(zip(name15630, type15630, latitude15630, longitude15630),
               columns =['name', 'type', 'latitude', 'longitude'])

In [458]:
df15630.drop_duplicates(inplace=True, ignore_index=True)
df15630

,name,type,latitude,longitude
0,Shell,gas_station,36.016064,-86.620556
1,bp,gas_station,36.015064,-86.618748
2,Family Mart,gas_station,36.022759,-86.584634
3,TA Travel Center,gas_station,36.014914,-86.618405
4,Exxon,gas_station,36.029188,-86.590429
...,...,...,...,...
92,XTRA Lease Nashville,point_of_interest,36.020798,-86.609920
93,Turner Seed Co,food,36.026241,-86.595486
94,Auto Central LLC.,car_dealer,36.031083,-86.611406
95,Shoe Metro,shoe_store,36.028400,-86.606337


In [459]:
df15630['tract'] = 'census_tract_15630'
df15630

,name,type,latitude,longitude,tract
0,Shell,gas_station,36.016064,-86.620556,census_tract_15630
1,bp,gas_station,36.015064,-86.618748,census_tract_15630
2,Family Mart,gas_station,36.022759,-86.584634,census_tract_15630
3,TA Travel Center,gas_station,36.014914,-86.618405,census_tract_15630
4,Exxon,gas_station,36.029188,-86.590429,census_tract_15630
...,...,...,...,...,...
92,XTRA Lease Nashville,point_of_interest,36.020798,-86.609920,census_tract_15630
93,Turner Seed Co,food,36.026241,-86.595486,census_tract_15630
94,Auto Central LLC.,car_dealer,36.031083,-86.611406,census_tract_15630
95,Shoe Metro,shoe_store,36.028400,-86.606337,census_tract_15630


In [460]:
location_162= '36.13994, -86.78199'
radius_162= '1854'

In [461]:
ct162 = []
ct162_list = []
latitude162 = []
longitude162 = []
name162 = []
type162 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct162_res = search_places_by_coordinate(location_162, radius_162, i, api_key) # running the API function through each type
    ct162.append(ct162_res) # creating a lists of the lists of each type of business
    for list in ct162:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude162.append(element['geometry']['location']['lat'])
            longitude162.append(element['geometry']['location']['lng'])
            name162.append(element['name'])
            type162.append(element['types'][0])

In [462]:
df162 = pd.DataFrame(zip(name162, type162, latitude162, longitude162),
               columns =['name', 'type', 'latitude', 'longitude'])

In [463]:
df162.drop_duplicates(inplace=True, ignore_index=True)
df162

,name,type,latitude,longitude
0,MAPCO Mart,gas_station,36.134648,-86.779522
1,Shell,gas_station,36.132442,-86.779478
2,Exxon,gas_station,36.132638,-86.795786
3,bp,gas_station,36.134567,-86.779460
4,Circle K,convenience_store,36.132559,-86.795966
...,...,...,...,...
265,Music City Optical,doctor,36.149909,-86.792242
266,Hanging Around,point_of_interest,36.138175,-86.778148
267,Carnival Music,electronics_store,36.150224,-86.793008
268,Movie Music Inc.,point_of_interest,36.149146,-86.793800


In [464]:
df162['tract'] = 'census_tract_162'
df162

,name,type,latitude,longitude,tract
0,MAPCO Mart,gas_station,36.134648,-86.779522,census_tract_162
1,Shell,gas_station,36.132442,-86.779478,census_tract_162
2,Exxon,gas_station,36.132638,-86.795786,census_tract_162
3,bp,gas_station,36.134567,-86.779460,census_tract_162
4,Circle K,convenience_store,36.132559,-86.795966,census_tract_162
...,...,...,...,...,...
265,Music City Optical,doctor,36.149909,-86.792242,census_tract_162
266,Hanging Around,point_of_interest,36.138175,-86.778148,census_tract_162
267,Carnival Music,electronics_store,36.150224,-86.793008,census_tract_162
268,Movie Music Inc.,point_of_interest,36.149146,-86.793800,census_tract_162


In [465]:
location_193= '36.17114, -86.76886'
radius_193= '2977'

In [466]:
ct193 = []
ct193_list = []
latitude193 = []
longitude193 = []
name193 = []
type193 = [] # create some empty lists to iterate through and then to create my ending dataframe

for i in all_types:
    ct193_res = search_places_by_coordinate(location_193, radius_193, i, api_key) # running the API function through each type
    ct193.append(ct193_res) # creating a lists of the lists of each type of business
    for list in ct193:
        for element in list: # iterating through each list of businesses to get the info we need for the dataframe
            latitude193.append(element['geometry']['location']['lat'])
            longitude193.append(element['geometry']['location']['lng'])
            name193.append(element['name'])
            type193.append(element['types'][0])

In [467]:
df193 = pd.DataFrame(zip(name193, type193, latitude193, longitude193),
               columns =['name', 'type', 'latitude', 'longitude'])

In [468]:
df193.drop_duplicates(inplace=True, ignore_index=True)
df193

,name,type,latitude,longitude
0,TA Travel Center,gas_station,36.171075,-86.771986
1,Trillium,gas_station,36.178444,-86.774921
2,Shell,gas_station,36.171552,-86.751873
3,Express Food Mart,convenience_store,36.177783,-86.749592
4,Marathon,gas_station,36.177779,-86.749787
...,...,...,...,...
224,Jane Langford Designs,home_goods_store,36.170942,-86.763261
225,Island Bound Music,electronics_store,36.162543,-86.775041
226,Dr. Angela Walk,point_of_interest,36.163374,-86.778357
227,Patterson Pope,furniture_store,36.173158,-86.764914


In [469]:
df193['tract'] = 'census_tract_193'
df193

,name,type,latitude,longitude,tract
0,TA Travel Center,gas_station,36.171075,-86.771986,census_tract_193
1,Trillium,gas_station,36.178444,-86.774921,census_tract_193
2,Shell,gas_station,36.171552,-86.751873,census_tract_193
3,Express Food Mart,convenience_store,36.177783,-86.749592,census_tract_193
4,Marathon,gas_station,36.177779,-86.749787,census_tract_193
...,...,...,...,...,...
224,Jane Langford Designs,home_goods_store,36.170942,-86.763261,census_tract_193
225,Island Bound Music,electronics_store,36.162543,-86.775041,census_tract_193
226,Dr. Angela Walk,point_of_interest,36.163374,-86.778357,census_tract_193
227,Patterson Pope,furniture_store,36.173158,-86.764914,census_tract_193


## Now I will make a list of all the dataframes and concatenate them into one.

In [470]:
all_tracts = [df10105,
              df10106,
              df10903,
              df10904,
              df11001,
              df114,
              df118,
              df119,
              df126,
              df12701,
              df12702,
              df12801,
              df12802,
              df13601,
              df13602,
              df137,
              df138,
              df139,
              df142,
              df143,
              df144,
              df148,
              df15630,
              df162,
              df193]

In [471]:
businesses_pred_black_tracts = pd.concat(all_tracts)

In [472]:
businesses_pred_black_tracts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3816 entries, 0 to 228
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       3816 non-null   object 
 1   type       3816 non-null   object 
 2   latitude   3816 non-null   float64
 3   longitude  3816 non-null   float64
 4   tract      3816 non-null   object 
dtypes: float64(2), object(3)
memory usage: 178.9+ KB


## Since I ran the API multiple times and used a buffer, I may have some duplicate businesses across tracts. I want to drop duplicates from the entire dataframe. 

In [473]:
unique_businesses  = businesses_pred_black_tracts.drop_duplicates(
  subset = ['latitude', 'longitude', 'name'],
  keep = 'first').reset_index(drop = True)

In [474]:
unique_businesses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2140 entries, 0 to 2139
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       2140 non-null   object 
 1   type       2140 non-null   object 
 2   latitude   2140 non-null   float64
 3   longitude  2140 non-null   float64
 4   tract      2140 non-null   object 
dtypes: float64(2), object(3)
memory usage: 83.7+ KB


## I will save this as a csv. In another notebook I will turn it into a GeoPandasDataFrame and merge it with the race GPD so we can use it for some map visualizations.

In [475]:
unique_businesses.to_csv('../data/businesses_in_pred_black_tracts.csv')